In [319]:
import numpy as np
import pandas as pd
from collections import defaultdict, deque, OrderedDict
#from tqdm import tqdm
from tqdm import tqdm_notebook as tqdm
from sklearn.preprocessing import normalize

In [2]:
df = pd.read_csv('smalltrace.csv', sep='\t')
df.columns = ['no','timestamp','pid','pname','bno', 'bsize', 'op', 'dvmajor', 'dvminor', 'blockhash']
df.head()

,no,timestamp,pid,pname,bno,bsize,op,dvmajor,dvminor,blockhash
0,0,89966811921625,4257,nfsd,488846200,8,W,6,0,2c4787c7671c2f638777c4ab72859ce1
1,1,89967003980393,4253,nfsd,508455704,8,W,6,0,88b93b628d84082186026d9da044f173
2,2,89967004049935,4253,nfsd,508455712,8,W,6,0,b5e9f4e5ab62a4fff5313a606b0ad4e3
3,3,89967004093615,4253,nfsd,508455720,8,W,6,0,7af5556ad4d45263bcd8358867333dd4
4,4,89967324150054,4254,nfsd,508500288,8,W,6,0,88b93b628d84082186026d9da044f173


In [313]:
blocktrace = df[['timestamp', 'bno']].apply(pd.to_numeric)
blocktrace.head()

,timestamp,bno
0,89966811921625,488846200
1,89967003980393,508455704
2,89967004049935,508455712
3,89967004093615,508455720
4,89967324150054,508500288


In [447]:
def getFurthestAccessBlock(C, OPT):
    maxAccessPosition = -1
    maxAccessBlock = -1
    for cached_block in C:
        if len(OPT[cached_block]) is 0:
            return cached_block        
    for cached_block in C:
        if OPT[cached_block][0] > maxAccessPosition:
            maxAccessPosition = OPT[cached_block][0]
            maxAccessBlock = cached_block
    return maxAccessBlock

def belady_opt(blocktrace, frame):
    
    OPT = defaultdict(deque)
    
    for i, block in tqdm(blocktrace.iterrows(), desc="OPT: building index"):
        OPT[block['bno']].append(i)    
    
    
    hit, miss = 0, 0
    C = deque()
    evictBlock = deque()
    time = deque()
    timestamp = OrderedDict()
    for i,block in tqdm(blocktrace.iterrows(), desc="OPT"):
        OPT[block['bno']].popleft()

        if block['bno'] in C:
            hit+=1
        
        else:
            miss+=1
            if len(C) == frame:
                fblock = getFurthestAccessBlock(C, OPT)
                assert(fblock != -1)
                if miss % 500 == 0:
                    evictBlock.append(opt_wrapper(C, OPT))
                    time.append(normalize([list(timestamp.values())]).tolist())
                C.remove(fblock)
                del timestamp[fblock]
            C.append(block['bno'])
            timestamp[block['bno']] = block['timestamp']
            
                
    hitrate = hit / (hit + miss)
    print(hitrate)
    return time, evictBlock

In [448]:
def opt_wrapper(cache, OPT):
    
    evictlist = deque()
    evictBinary = deque([0]) * len(cache)
    cache2 = cache.copy()
    for i in range(int(len(cache) * 0.1)):
        evict = getFurthestAccessBlock(cache2, OPT)
        #print(evict)
        evictlist.append(evict)
        cache2.remove(evict)
    #print(evictlist)
    for i, j in enumerate(cache):
        if j in evictlist:
            evictBinary[i] = 1
    
    return evictBinary

In [449]:
time, evictBlock = belady_opt(blocktrace, 500)

0.0793


In [492]:
t = np.array(time)
t = t.flatten()
t = t.reshape(-1,1)
e = np.array(evictBlock)
e = e.flatten()
#e = e.reshape(-1,1)


In [493]:
t.shape

(8500, 1)

In [494]:
e.shape

(8500,)

In [386]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from functools import reduce

In [451]:
Xtime_ = reduce(lambda x,y: x+y, time)
Xtime = reduce(lambda x,y: x+y,Xtime_)

In [437]:
Ylabel = reduce(lambda x,y: x+y,evictBlock)

In [506]:
X_train, X_test, Y_train, Y_test = train_test_split(t, e ,test_size=0.3, shuffle=True)

In [507]:
clf = LogisticRegression().fit(X_train, Y_train)

In [508]:
accuracy_score(Y_test, clf.predict(X_test))

0.9003921568627451

In [509]:
Y_train.shape

(5950,)

In [510]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Y_test, clf.predict(X_test))

array([[2296,    0],
       [ 254,    0]])